In [1]:
import pandas as pd
from cluster_comparison import perform_umap, perform_hdbscan, calculate_silhouette
import random
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

/Users/leo/Programming/PLR/Leo/env/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/leo/Programming/PLR/Leo/env/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/leo/Programming/PLR/Leo/env/lib/python3.11/site-packages/umap/d

In [2]:
# Sample data and demographics DataFrames, replace these with your actual DataFrames
raw_data = pd.read_csv('/Users/leo/Programming/PLR/Leo/data/cleaned_data_SYMPTOMS_9_13_23_DNA.csv')
data_symp_groups = pd.read_csv('data/skew_corr_groupadd.csv', usecols=['Grouped_Neuro_Sensory', 'Grouped_Cognitive_Memory', 'Grouped_Gastrointestinal', 'Grouped_Respiratory_Cardiac', 'Grouped_Eye_Vision'])
data_symp_groups_all = pd.read_csv('data/skew_corr_groupadd.csv')
demographics = pd.read_csv('/Users/leo/Programming/PLR/Leo/data/non_binary_data_processed.csv')

# combine demographics and data_symp_group_all 
demo_all = pd.concat([demographics, data_symp_groups_all], axis=1).drop(['Unnamed: 0'], axis=1)

# combine demographics and data_symp_group
demo_groups = pd.concat([demographics, data_symp_groups], axis=1)

In [3]:
print(demo_all.shape)

(6031, 141)


In [4]:
# Datasets to consider
datasets = {'data_symp_groups_all': demo_all}
print(f"number of features: {len(datasets['data_symp_groups_all'].columns)}")

# Genetic Algorithm Parameters
population_size = 20
n_generations = 100
selection_rate = 0.3
mutation_rate = 0.05

# Elitism: number of top individuals to directly carry forward to the next generation
num_elites = int(0.1 * population_size)

# Add this dictionary at the top of your code to cache fitness results
fitness_cache = {}

# Initialize variables for adaptive mutation
last_best_fitness = -1
increased_mutation_rate = 0.2
n_features = len(datasets['data_symp_groups_all'].columns)

# Fitness function
def fitness(params):
    # Convert params to a tuple since lists are unhashable
    params_tuple = tuple(params)

    # Check if this individual is in the cache
    if params_tuple in fitness_cache:
        return fitness_cache[params_tuple]

    selected_features = params[:n_features]
    n_neighbors, min_dist, min_cluster_size, n_components = params[n_features:]

    dataset_name, dataset = random.choice(list(datasets.items()))
    dataset = dataset[[col for col, keep in zip(dataset.columns, selected_features) if keep]].dropna()
    n_neighbors = max(2, int(n_neighbors))
    min_cluster_size = max(2, int(min_cluster_size))
    n_components = max(2, int(n_components))
    min_dist = min(min_dist, 1.0)

    umap_result = perform_umap(dataset, n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components)
    labels = perform_hdbscan(umap_result, min_cluster_size=min_cluster_size)
    score = calculate_silhouette(umap_result, labels)
    
    # Save the computed score in the cache
    fitness_cache[params_tuple] = score

    return score

# Initialize population
population = []
for _ in range(population_size // 4):
    feature_vector = tuple(random.choices([0, 1], k=n_features))

    # First set of individuals (close to best performing genome)
    population.append(feature_vector + (0.6492373060806188, 0.03733592291556887, 29, 47))
    population.append(feature_vector + (0.6757706356698308, 0.04030749239323905, 38, 35))
    
    # Second set of individuals (random within a specific range)
    population.append(feature_vector + (random.uniform(0.1, 5), random.uniform(0.045, 0.75), random.randint(30, 45), random.randint(8, 22)))
    population.append(feature_vector + (random.uniform(6, 20), random.uniform(0.76, 1.0), random.randint(46, 50), random.randint(23, 25)))
    
    # Third set of individuals (another set close to best performing genome but with slight variations)
    population.append(feature_vector + (0.6757706356698308 + random.uniform(-0.1, 0.1), 0.04030749239323905 + random.uniform(-0.01, 0.01), 29 + random.randint(-2, 2), 48 + random.randint(-2, 2)))
    population.append(feature_vector + (0.6757706356698308 + random.uniform(-0.1, 0.1), 0.04030749239323905 + random.uniform(-0.01, 0.01), 38 + random.randint(-2, 2), 35 + random.randint(-2, 2)))

    # Fourth set of individuals (another set of random individuals)
    population.append(feature_vector + (random.uniform(21, 35), random.uniform(0.26, 0.5), random.randint(21, 35), random.randint(25, 38)))
    population.append(feature_vector + (random.uniform(36, 45), random.uniform(0.51, 0.75), random.randint(36, 45), random.randint(39, 65)))

# To store best fitness and variance for each generation
best_fitnesses = []
variances = []

# Main GA loop
for generation in tqdm(range(n_generations), desc="Generations"):
    scores = Parallel(n_jobs=-1)(delayed(fitness)(ind) for ind in population)
    
    best_fitness = max(scores)
    best_fitnesses.append(best_fitness)
    variances.append(np.var(scores))
    
    if best_fitness <= last_best_fitness:
        mutation_rate = increased_mutation_rate
    else:
        mutation_rate = 0.1
    last_best_fitness = best_fitness

    # Sort population by fitness
    sorted_indices = np.argsort(scores)[::-1]
    elites = [population[i] for i in sorted_indices[:num_elites]]

    fitness_sum = sum(scores)
    selected_population = []
    for _ in range(int(selection_rate * population_size)):
        pick = random.uniform(0, fitness_sum)
        current = 0
        for i in range(len(scores)):
            current += scores[i]
            if current > pick:
                selected_population.append(population[i])
                break

    children = []
    while len(children) < population_size - len(selected_population) - 1:
        parent1, parent2 = random.sample(selected_population, 2)
        crossover_point = random.randint(1, len(parent1) - 1)
        child = parent1[:crossover_point] + parent2[crossover_point:]
        children.append(child)

    mutations = 0
    for i in range(len(children)):
        if random.random() < mutation_rate:
            mutate_pos = random.randint(0, len(children[i]) - 1)
            if mutate_pos < n_features:
                mutated_bit = 1 - children[i][mutate_pos]
                children[i] = children[i][:mutate_pos] + (mutated_bit,) + children[i][mutate_pos+1:]
            else:
                new_value = random.choice([
                    random.randint(5, 50),
                    random.uniform(0.0, 1.0),
                    random.randint(5, 50),
                    random.randint(2, 25)
                ])
                children[i] = children[i][:mutate_pos] + (new_value,) + children[i][mutate_pos+1:]
            mutations += 1
    print(f"Number of mutations: {mutations}")

    # Update the population for the next generation
    population = elites + selected_population + children

    # Printing the elite individuals at the end of the loop for the generation
    print(f"\nElite individuals in generation {generation+1}:")
    for i, elite in enumerate(elites):
        elite_idx = sorted_indices[i]
        print(f"  Genome: {elite}, Fitness: {scores[elite_idx]}")

final_scores = Parallel(n_jobs=-1)(delayed(fitness)(ind) for ind in population)
best_idx = max(range(len(final_scores)), key=lambda i: final_scores[i])
best_individual = population[best_idx]

print("\nBest parameters found:")
print(f"Selected Features: {best_individual[:n_features]}")
print(f"n_neighbors={best_individual[n_features]}, min_dist={best_individual[n_features+1]}, min_cluster_size={best_individual[n_features+2]}, n_components={best_individual[n_features+3]}")
print(f"Best silhouette score: {final_scores[best_idx]}")

plt.figure(figsize=(10, 6))
plt.plot(range(1, n_generations + 1), best_fitnesses, label='Best Fitness')
plt.fill_between(range(1, n_generations + 1), 
                 np.array(best_fitnesses) - np.array(variances), 
                 np.array(best_fitnesses) + np.array(variances), 
                 color='gray', alpha=0.5, label='Variance')
plt.xlabel('Generation')
plt.ylabel('Fitness')
plt.title('Evolution of Best Fitness Over Generations')
plt.legend()
plt.show()


number of features: 141


Generations:   0%|          | 0/100 [00:00<?, ?it/s]

/Users/leo/Programming/PLR/Leo/env/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/leo/Programming/PLR/Leo/env/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/leo/Programming/PLR/Leo/env/lib/python3.11/site-packages/umap/d

Number of mutations: 2

Elite individuals in generation 1:
  Genome: (0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0.6492373060806188, 0.03733592291556887, 29, 47), Fitness: 0.8448787331581116
  Genome: (0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0

/Users/leo/Programming/PLR/Leo/env/lib/python3.11/site-packages/umap/spectral.py:260: UserWarning: WARNING: spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(
Generations:   2%|▏         | 2/100 [01:39<1:17:31, 47.47s/it]

...

Number of mutations: 1

Elite individuals in generation 69:
  Genome: (0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0.6492373060806188, 0.03733592291556887, 29, 47), Fitness: 0.8958989381790161
  Genome: (0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 

Generations:  70%|███████   | 70/100 [34:06<15:10, 30.36s/it]

Number of mutations: 1

Elite individuals in generation 70:
  Genome: (0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0.6492373060806188, 0.03733592291556887, 29, 47), Fitness: 0.8980623483657837
  Genome: (0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 

Generations:  71%|███████   | 71/100 [34:35<14:27, 29.92s/it]

Number of mutations: 1

Elite individuals in generation 71:
  Genome: (0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0.6492373060806188, 0.03733592291556887, 29, 47), Fitness: 0.9132695198059082
  Genome: (0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 